# Pretrained model

In [12]:
model_name = "TinyLLaMA/TinyLlama-1.1B-Chat-v1.0"

# Prepare a dataset

In [1]:
import pandas as pd
from datasets import Dataset

data = {
    "instruction": [
        "Explain what machine learning is.",
        "What is NLP?",
        "Write a short note on transformers.",
        "Describe supervised learning.",
    ],
    "response": [
        "Machine learning is a field of AI that enables computers to learn patterns from data.",
        "Natural Language Processing (NLP) is a field that focuses on enabling machines to understand human language.",
        "Transformers are deep learning models that use self-attention to process sequences.",
        "Supervised learning is when a model is trained on labeled data to make predictions."
    ]
}




/Users/crtejavardhanreddy/Desktop/finetuning/fine_tuning/env/lib/python3.9/site-packages/urllib3/__init__.py:35: NotOpenSSLWarning: urllib3 v2 only supports OpenSSL 1.1.1+, currently the 'ssl' module is compiled with 'LibreSSL 2.8.3'. See: https://github.com/urllib3/urllib3/issues/3020
  warnings.warn(
/Users/crtejavardhanreddy/Desktop/finetuning/fine_tuning/env/lib/python3.9/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
df = pd.DataFrame(data)

In [3]:
df

,instruction,response
0,Explain what machine learning is.,Machine learning is a field of AI that enables...
1,What is NLP?,Natural Language Processing (NLP) is a field t...
2,Write a short note on transformers.,Transformers are deep learning models that use...
3,Describe supervised learning.,Supervised learning is when a model is trained...


In [4]:
dataset = Dataset.from_pandas(df)
dataset

Dataset({
    features: ['instruction', 'response'],
    num_rows: 4
})

# Format for tuning

In [5]:
def format_chat(example):
    prompt = f"User: {example['instruction']}\n Assistant: {example['response']}"
    return {"text":prompt}

In [6]:
dataset = dataset.map(format_chat)

Map: 100%|██████████| 4/4 [00:00<00:00, 605.50 examples/s]


In [7]:
dataset

Dataset({
    features: ['instruction', 'response', 'text'],
    num_rows: 4
})

In [8]:
dataset['text']

Column(['User: Explain what machine learning is.\n Assistant: Machine learning is a field of AI that enables computers to learn patterns from data.', 'User: What is NLP?\n Assistant: Natural Language Processing (NLP) is a field that focuses on enabling machines to understand human language.', 'User: Write a short note on transformers.\n Assistant: Transformers are deep learning models that use self-attention to process sequences.', 'User: Describe supervised learning.\n Assistant: Supervised learning is when a model is trained on labeled data to make predictions.'])

In [9]:
from transformers import AutoTokenizer, AutoModelForCausalLM
import torch

In [13]:
tokenizer = AutoTokenizer.from_pretrained(model_name)

In [15]:
tokenizer.pad_token = tokenizer.eos_token


In [17]:
from transformers import AutoModelForCausalLM, BitsAndBytesConfig

bnb_config = BitsAndBytesConfig(
    load_in_4bit=True,
    bnb_4bit_compute_dtype="float16",
    bnb_4bit_use_double_quant=True
)

model = AutoModelForCausalLM.from_pretrained(
    model_name,
    quantization_config=bnb_config,
    dtype="float16",
    device_map="auto"
)


ImportError: The installed version of bitsandbytes (<0.43.1) requires CUDA, but CUDA is not available. You may need to install PyTorch with CUDA support or upgrade bitsandbytes to >=0.43.1.